# LSST Catalog Access Tutorial

This notebook provides a simple tutorial for how to access the catalog data for LSST.

We use the IVOA TAP (Table Access Procotol) standard to access the catalog data.  This standardizes usage so that 3rd party clients and astronomers can easily move their queries around between datasets, and discover what data is available for them to query.

Here's a link to the standard: http://www.ivoa.net/documents/TAP/

The TAP service uses a query language similar to SQL called ADQL.  For the specifics about the syntax and keywords, look at: http://www.ivoa.net/documents/latest/ADQL.html

### TOPCAT usage

Some astronomers prefer to use tools like TOPCAT to access the catalogs.  If you can contact http://lsst-lsp-stable.ncsa.illinois.edu/api/tap/ from your network, you can also use this as the "Selected TAP Service" in TOPCAT.

**NOTE**: Most people won't have internet access to NCSA.  If you are on your institution's network (such as NOAO) you may have access.  Otherwise you will have to use the Cisco VPN client to connect to the network first, then you can use this URL.

### Portal usage

The TAP service is used by the portal and users can use the Portal UI to help them construct queries.  For more information look at: **TODO find link**

### Notebook Usage

This notebook assumes you are running it in the notebook aspect of the LSST Science Platform at NCSA (https://lsst-lsp-stable.ncsa.illinois.edu/nb).

In the following steps, we will create a client to query the catalog, show how you can discover what tables and columns exist in the catalog, as well as how to run queries and process their results using python.

For this example, we will be using pyvo: https://pyvo.readthedocs.io/en/latest/index.html

## 1. Create the client

First, you must create a pyvo object to use for querying the data.  This will be the object you call to run queries on.  Run the cell below to create your client.

In [ ]:
import pandas
import pyvo
pandas.set_option('display.max_rows', 1000)
service = pyvo.dal.TAPService('http://lsst-lsp-stable.ncsa.illinois.edu/api/tap')

# 2. Schema discovery

Now that you've connected to the TAP service, you need to know the names of schemas, tables, and columns (and their datatypes) to be able to write queries to access the data.

Luckily, TAP provides for a standard way of discovering what schemas, tables, and columns are available.  This information is stored in a database named TAP_SCHEMA.

### 2.1 What schemas can I query?

In order to discover what database schemas are being served and can be queried against, you can run the following cell.  You should see a numpy compatible table printed out in the notebook that has two important columns, description and schema_name.  

In [ ]:
results = service.search("SELECT schema_name, description FROM TAP_SCHEMA.schemas")
results.to_table().show_in_notebook()

**TAP_SCHEMA.schemas** behaves like a normal SQL table, with mostly the same query language. Just like standard SQL, you can also retrieve all the columns by using the following example.

**TIP:** Calling to_table() on a result will give you an AstroPy table (http://docs.astropy.org/en/stable/_modules/astropy/table/table.html).

From an astropy table, you can also convert to a pandas dataframe by using to_pandas().

In [ ]:
results = service.search("SELECT * FROM TAP_SCHEMA.schemas")
results.to_table().show_in_notebook()

### 2.2 What tables can I query?

Like the TAP_SCHEMA.schemas table, there is also a **TAP_SCHEMA.tables** table, which contains the names of the tables, and which schema each table belongs to.

In [ ]:
results = service.search("SELECT * FROM TAP_SCHEMA.tables")
results.to_table().show_in_notebook()

Sometimes, you may want to act programmatically on the results.  There are also functions you can call on the results object to get the columns / fields, and iterate through the rows, as demonstrated below.

In [ ]:
print("Column names for TAP_SCHEMA.tables are:")
for field in results.fieldnames:
    print(field)

We can now use the column names to limit the columns returned by the query, as seen below.

In [ ]:
results = service.search("SELECT schema_name, table_name FROM TAP_SCHEMA.tables")
results.to_table().show_in_notebook()

### 2.3 What columns are in a table, and what datatype is each column?

Now that we've learned what tables exist, we're close to querying for real catalog data.  But generally, we want to limit the number of columns to return to exactly what we need to get the job done.  This generally makes things faster and better for everyone.  While you can get all the columns that exist (with a "SELECT * from TAP_SCHEMA.columns"), this will be over all the tables in all the schemas, so normally we want to limit the query to a particular table.

Let's discover what columns exist in the wise_00.allwise_p3as_mep table using the following code.

**TIP**: Be careful with your quoting, since some quotes are processed by python, and some quotes are passed down as a part of the query.  My suggestion is to use double quotes for python, and single quotes for SQL.

In [ ]:
results = service.search("SELECT * from TAP_SCHEMA.columns WHERE table_name = 'wise_00.allwise_p3as_mep'")

print("Column names for TAP_SCHEMA.columns are:")
print(results.fieldnames)

print("Columns that exist in wise_00.allwise_p3as_mep are:")
results.to_table().show_in_notebook()

The most useful columns in TAP_SCHEMA.columns are the column_name, unit, and description.  Let's just query those.

In [ ]:
results = service.search("SELECT column_name, description, unit FROM TAP_SCHEMA.columns WHERE table_name = 'wise_00.allwise_p3as_mep'")
results.to_table().show_in_notebook()

# 3. Example queries

Here are some example queries against the wise dataset.  As more queries are supported, they will be added here.

**Note**: Not all of these are against the same table, or against the same columns.  These queries are simply for reference.

### 3.1 Cone searches

How to do a cone search (with a center RA/DEC, and a radius):

The format is CONTAINS(POINT('ICRS', dec column or number, ra column or number), CIRCLE('ICRS', center RA, center DEC, radius)) = 1.  All units are in degrees.

**TIP**: You can limit the number of rows returned by using maxrec=x as a parameter.  This means it will only return x rows.

In [ ]:
results = service.search("SELECT ra, decl FROM wise_00.allwise_p3as_mep WHERE CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 1.0, -1.0, .1)) = 1", maxrec=10)
results.to_table().show_in_notebook()

### 3.2 Polygon searches

Sometimes cone search isn't enough.  Sometimes you want to submit a set of arbitrary vertices and draw your own polygon to limit to.  You can use the POLYGON function.  This behaves like CIRCLE, but allows for any number of RA/DEC coordinate pairs to be vertices.  The polygon is automatically closed between the last vertex and the first vertex.

The format is CONTAINS(POINT('ICRS', dec column or number, ra column or number), POLYGON('ICRS', RA 1, DEC 1, RA 2, DEC 2, ...)) = 1. All units are in degrees.

Following is an example of using POLYGON to make a square like shape.

In [ ]:
results = service.search("SELECT ra, decl FROM wise_00.allwise_p3as_mep WHERE CONTAINS(POINT('ICRS', ra, decl), POLYGON('ICRS', .9, .9, 1, .9, 1, 1, .9, 1)) = 1", maxrec=10)
results.to_table().show_in_notebook()

### 3.3 Retrieve a light curve for a source

In [ ]:
results = service.search("SELECT mjd, w1mpro_ep, w1sigmpro_ep, w2mpro_ep, w2sigmpro_ep FROM wise_00.allwise_p3as_mep WHERE source_id_mf = '2813m031_ac51-041856' AND mjd > 55450")
results.to_table().show_in_notebook()

Now you can convert the data to a pandas dataframe, and graph it using Bokeh (https://bokeh.pydata.org/en/latest/docs/user_guide.html).

**Tip** Note some of the columns don't have data.  You always have to watch out for NULLs or missing data in your code.  This can be really tricky!

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Whisker
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from jupyterlabutils import show_with_bokeh_server

output_notebook(INLINE, hide_banner=True)

dataframe = results.to_table().to_pandas()

# Let's calculate the error bars.
dataframe['w1mpro_err_min'] = dataframe['w1mpro_ep'] - dataframe['w1sigmpro_ep']
dataframe['w1mpro_err_max'] = dataframe['w1mpro_ep'] + dataframe['w1sigmpro_ep']
dataframe['w2mpro_err_min'] = dataframe['w2mpro_ep'] - dataframe['w2sigmpro_ep']
dataframe['w2mpro_err_max'] = dataframe['w2mpro_ep'] + dataframe['w2sigmpro_ep']

datasource = ColumnDataSource(dataframe)

# Make a plot of a certain size
p = figure(plot_width=800, plot_height=800)
p.xaxis.axis_label = 'Modified Julian date of the mid-point of the observation of the frame'
p.yaxis.axis_label = 'Single-exposure profile-fit magnitude'

# Let's add the error bars now
p.add_layout(Whisker(source=datasource, base='mjd', upper='w1mpro_err_max', lower='w1mpro_err_min'))
p.add_layout(Whisker(source=datasource, base='mjd', upper='w2mpro_err_max', lower='w2mpro_err_min'))

# Add a circle renderer with a size, color, and alpha.
# x and y control which columns to use from the dataframe.
p.circle(x='mjd', y='w1mpro_ep', source=datasource, size=5, color="navy")
p.circle(x='mjd', y='w2mpro_ep', source=datasource, size=5, color="green")

# show the results
show_with_bokeh_server(p)

# 4. Future work / Known issues

The work for this service is ongoing, and there will be known limitations and bugs.  If you want to contact someone for help, please go to the #dm-dax channel on the LSST Slack or email Christine Banek (cbanek@lsst.org).  You can also file an issue in JIRA and assign it to cbanek.

### 4.1 Known limitations

This TAP layer is built on top of QServ, which has its own query limitations that may come up.  Here is a document outlining those restrictions:

Some ADQL functions are not yet supported:

- AREA
- BOX
- COORDSYS
- COORD1
- COORD2
- DISTANCE
- INTERSECTS

The only coordinate system currently supported is 'ICRS'.

https://github.com/lsst/qserv/blob/master/UserManual.md

### 4.2 Performance

If your query takes more than a minute or so, it's likely you have run a query that is enacting a full table scan.  This can take a long time (~ hour) to return.  Please be patient and don't re-run your query as this may put the database under additional load.